In [2]:
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np

In [3]:
mat = loadmat('data/S06.mat')
panel = pd.Panel(mat['data'])
''' Item axis: Epoch
    Major axis: Electrode
    Minor axis: Sample '''
panel

FileNotFoundError: [Errno 2] No such file or directory: 'data/S06.mat'

In [ ]:
FREQ = 250 # Hz
ALPHA_RELEVANT_ELECTRODES = [8,44,80,131,185]

def plot_electrode(x):
    plt.plot(np.linspace(-200,600,201), x, alpha=0.5)

def plot_welch(x):
    f, power = signal.welch(x, fs=FREQ, nperseg=len(x), nfft=2048)
    plt.plot(f, power, '-')
    #plt.yscale('log')
    plt.xlim(-2,45)
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('PSD [V^2/Hz]')

# Un solo Epoch

In [ ]:
for electrode in ALPHA_RELEVANT_ELECTRODES:
    plot_electrode(panel[0,electrode])
plt.show()

plt.figure(figsize=(13, 8))
for electrode in ALPHA_RELEVANT_ELECTRODES:
    plot_welch(panel[0,electrode])
plt.show()

# Promedio entre Epochs

In [ ]:
def mean_between_epochs(panel):
    return panel.mean(0)

mean_df = mean_between_epochs(panel).T

for electrode in ALPHA_RELEVANT_ELECTRODES:
    plot_electrode(mean_df[electrode])
plt.show()

plt.figure(figsize=(13, 8))
for electrode in ALPHA_RELEVANT_ELECTRODES:
    plot_welch(mean_df[electrode])
plt.show()


In [ ]:
def mean_between_electrodes(panel, electrodes):
    return panel[:,electrodes].mean(1)

def fourier_per_epoch(panel, electrodes):
    samples_per_epoch = mean_between_electrodes(panel, electrodes)
    relevant_frecuencies = 38
    res = []
    for _, samples in samples_per_epoch.iteritems():
        frequencies, powers = signal.welch(samples, FREQ, nperseg=len(samples))
        res.append(powers[:relevant_frecuencies])
    return np.asarray(res), frequencies[:relevant_frecuencies]

spectrogram_alpha, frequencies_alpha = fourier_per_epoch(panel, ALPHA_RELEVANT_ELECTRODES)
#spectrogram_all, frequencies_all = fourier_per_epoch(panel, range(256))

In [ ]:
def plot_spectrogram(spectrogram, frequencies):
    spectrogram = spectrogram.T
    plt.figure(figsize=(13, 8))
    ax = sns.heatmap(spectrogram, 
                     xticklabels=100, yticklabels=frequencies.round(2), 
                     #cbar_kws=('PSD [V**2/Hz]'),
                     vmin=0.1e-11, vmax=1e-11, 
                     #center=1.5e-11,
                     cmap='YlGnBu_r'
                     )
    plt.title('Promedios de potencia de frecuencias de varios electrodos [V^2/Hz]')
    plt.xlabel('Epoch')
    plt.ylabel('Frequency (Hz)')
    plt.show()

plot_spectrogram(spectrogram_alpha, frequencies_alpha)

# Categorizamos las frecuencias según su tipo

In [ ]:
def mean_power_per_frequency_range(panel):
    mean_samples = panel.mean(0).mean(0)

    def belongs(frequency_range, frequency):
        return frequency_range[0] <= frequency < frequency_range[1]

    frequencies, power = signal.welch(mean_samples, fs=FREQ, nperseg=len(mean_samples), nfft=2048)
    #plt.plot(frequencies, power)
    #plt.show()
    limits = {'delta': (0,4), 'tita': (4,8), 'alpha': (8,13), 'beta': (13,30), 'gamma': (30, 45)}
    values_by_range = {'delta':[], 'tita':[], 'alpha':[], 'beta':[], 'gamma':[]}

    for f, p in zip(frequencies, power):
        for key, limit in limits.items():
            if belongs(limit, f):
                values_by_range[key].append(p)

    return {key: np.mean(value) for key, value in values_by_range.items()}

In [ ]:
import gc
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('data') if (f.endswith('.mat') and f.startswith('S'))]

powers_per_frequency_range_S = []
df = {}
for filename in onlyfiles:
    mat = loadmat(join('data', filename))
    panel = pd.Panel(mat['data'])
    mean = mean_power_per_frequency_range(panel)
    df[filename, 'S'] = mean
    #mean['type'] = 'S'
    powers_per_frequency_range_S.append(mean)
    del mat
    del panel
    gc.collect()
pd.DataFrame(df)

In [ ]:
onlyfiles = [f for f in listdir('data') if (f.endswith('.mat') and f.startswith('P'))]
powers_per_frequency_range_P = []
df = {}
for filename in onlyfiles:
    mat = loadmat(join('data', filename))
    panel = pd.Panel(mat['data'])
    mean = mean_power_per_frequency_range(panel)
    #print(mean)
    df[filename,'P'] = mean
    #mean['type'] = 'P'
    powers_per_frequency_range_P.append(mean)
    del mat
    del panel
    gc.collect()
pd.DataFrame(df)

In [ ]:
dS = pd.DataFrame(powers_per_frequency_range_S)
dP = pd.DataFrame(powers_per_frequency_range_P)
data = pd.concat([dS,dP])
print(data.shape)
print(data)

data = data.rename_axis("patient", axis=0)
data = data.rename_axis("band", axis=1)


ax = sns.stripplot(data=data, 
                   vmin=data.min().min(), 
                   vmax=data.max().max()
                  )
#ax.set_yscale('log')
plt.title('Distribution per band')
plt.show()

df = data
df_norm = (df - df.min().min()) / (df.max().max() - df.min().min())
ax = sns.stripplot(data=df_norm, 
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
#ax.set_yscale('log')
plt.title('Distribution per band - Normalized')
plt.show()


df = data
df_norm = ( (df - df.min() ) / (df.max() - df.min()) )
ax = sns.stripplot(data=df_norm,
                   jitter=True,
                   vmin=df_norm.min().min(), 
                   vmax=df_norm.max().max() 
                  )
#ax.set_yscale('log')
plt.title('Distribution per band - Normalized by column')
plt.show()

# TODO: separar S y P en subcolumnas

## Realizamos todos los gráficos categóricos.
#### En ROJO los S, en AZUL los P.
Versión normalizados.

### Stripplot

In [ ]:
df_norm = (df - df.min().min()) / (df.max().max() - df.min().min())
ax = sns.stripplot(data=df_norm[:10],
                   color="r",
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
ax = sns.stripplot(data=df_norm[10:],
                   color="b",
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
#ax.set_yscale('log')
plt.title('Distribution per band - Normalized')
plt.show()

### Violin Plot

In [ ]:
df_norm = (df - df.min().min()) / (df.max().max() - df.min().min())
ax = sns.violinplot(data=df_norm[:10],
                   color="r",
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
ax = sns.violinplot(data=df_norm[10:],
                   color="b",
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
#ax.set_yscale('log')
plt.title('Distribution per band - Normalized')
plt.show()

### Bar Plot

In [ ]:
df_norm = (df - df.min().min()) / (df.max().max() - df.min().min())
ax = sns.barplot(data=df_norm[:10],
                   color="r",
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
ax = sns.barplot(data=df_norm[10:],
                   color="b",
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
#ax.set_yscale('log')
plt.title('Distribution per band - Normalized')
plt.show()

### SwarmPlot

In [ ]:
ax = sns.swarmplot(data=df_norm[:10],
                   color="r",
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
ax = sns.swarmplot(data=df_norm[10:],
                   color="b",
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
#ax.set_yscale('log')
plt.title('Distribution per band - Normalized')
plt.show()

### BoxPlot

In [ ]:
ax = sns.boxplot(data=df_norm[:10],
                   color="r",
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
ax = sns.boxplot(data=df_norm[10:],
                   color="b",
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
#ax.set_yscale('log')
plt.title('Distribution per band - Normalized')
plt.show()

### Point Plot

In [ ]:
ax = sns.pointplot(data=df_norm[:10],
                   color="r",
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
ax = sns.pointplot(data=df_norm[10:],
                   color="b",
                   vmin=df_norm.min(), 
                   vmax=df_norm.max() 
                  )
#ax.set_yscale('log')
plt.title('Distribution per band - Normalized')
plt.show()

## Gráficos categóricos

Elegimos usar de ahora en adelante el violin plot, ya que da un buen resúmen de la distribución de nuestros datos para cada categoría. 

## Análisis de información (intra-electrodo)

In [4]:
from scipy.io import loadmat
import pandas as pd

import gc
gc.collect()

mat = loadmat('data/S06.mat')
panel = pd.Panel(mat['data'])
''' Item axis: Epoch
    Major axis: Electrode
    Minor axis: Sample '''
min_ = panel.min().min().min()
max_ = panel.max().max().max()
bins = np.linspace(min_, max_, 100)

print(bins)

FileNotFoundError: [Errno 2] No such file or directory: 'data/S06.mat'

In [ ]:
# a) Computar una medida de información intra-electrodo. Calcular la media entre canales y epochs para cada sujeto. Realizar el gráfico elegido en el punto c) de la sección anterior, acompañado del test estadístico apropiado.

import scipy.stats
from collections import defaultdict
import numpy as np

import gc
gc.collect()

def compute_entropy(series):
    histogram, _ = np.histogram(series, bins)
    probabilities = [frecuency/len(series) for frecuency in histogram]
    return scipy.stats.entropy(probabilities)

def compute_entropies_for_subject(panel):
    n_epochs, n_electrodes, n_measures = panel.shape
    entropies = defaultdict(dict)
    for epoch in range(n_epochs):
        for electrode in range(n_electrodes):
            electrode_data = panel[epoch, electrode]
            entropy = compute_entropy(electrode_data)
            entropies[epoch][electrode] = entropy
    return pd.DataFrame(entropies)

In [ ]:
import matplotlib.pyplot as plt

def mean_between_epochs(df):
    return df.mean(0)
def mean_between_electrodes(df):
    return df.mean(1)

def plot_histograms_for_subject(entropies):
    # Promediamos por electrodos, obtenemos histograma por epochs
    mean_epochs = mean_between_epochs(entropies)
    print(mean_epochs.shape)
    mean_epochs.hist(bins=100)
    plt.axvline(mean_epochs.mean(), color='r')
    plt.xlim(0.7,1.7)
    plt.show()
    
    # Promediamos por epochs, obtenemos histograma por electrodos
    mean_electrodes = mean_between_electrodes(entropies)
    print(mean_electrodes.shape)
    mean_electrodes.hist(bins=100)
    plt.axvline(mean_electrodes.mean(), color='r')
    plt.xlim(0.7,1.7)
    plt.show()

def subject_global_mean(entropies):
    return entropies.mean().mean()

entropies = compute_entropies_for_subject(panel)
plot_histograms_for_subject(entropies)

In [ ]:
from os import listdir
from os.path import join

files_S = [f for f in listdir('data') if (f.endswith('.mat') and f.startswith('S'))]
files_P = [f for f in listdir('data') if (f.endswith('.mat') and f.startswith('P'))]
global_means_for_S = []
global_means_for_P = []

for filename in files_S:
    mat = loadmat(join('data', filename))
    panel = pd.Panel(mat['data'])
    entropies = compute_entropies_for_subject(panel)
    global_mean = subject_global_mean(entropies)
    print("Subject: {}, Entropy: {}".format(filename, global_mean))
    global_means_for_S.append(global_mean)
    del mat
    del panel
    gc.collect()
    
for filename in files_P:
    mat = loadmat(join('data', filename))
    panel = pd.Panel(mat['data'])
    entropies = compute_entropies_for_subject(panel)
    global_mean = subject_global_mean(entropies)
    print("Subject: {}, Entropy: {}".format(filename, global_mean))
    global_means_for_P.append(global_mean)
    del mat
    del panel
    gc.collect()

In [ ]:
plt.hist(global_means_for_S)
plt.hist(global_means_for_P)
plt.show()

## Análisis de información (inter-electrodo)

In [ ]:
import itertools
from functools import reduce

# TODO: revisar el bins global

def joint_entropy(signal_1, signal_2):
    _, bins_1 = np.histogram(signal_1, bins)
    _, bins_2 = np.histogram(signal_2, bins)
    
    signal_1 = np.digitize(signal_1, bins_1)
    signal_2 = np.digitize(signal_2, bins_2)
    
    signal = list(zip(signal_1, signal_2))
    probabilities = [signal.count(s) / len(signal) for s in signal]
    return scipy.stats.entropy(probabilities)
    

def inter_electrode_analysis(panel, epoch, electrode_1, electrode_2):
    signal_1 = panel[epoch, electrode_1]
    signal_2 = panel[epoch, electrode_2]
    return joint_entropy(signal_1, signal_2)

inter_electrode_analysis(panel, 0, 8, 24)

In [ ]:
from itertools import combinations
ALPHA_RELEVANT_ELECTRODES = [8,44,80,131,185]

def joint_entropy_for_subject(panel):
    entropies = []
    n_epochs, n_electrodes, n_measures = panel.shape
    for epoch in range(n_epochs):
        for combination in combinations(ALPHA_RELEVANT_ELECTRODES, 2):
            entropies.append(inter_electrode_analysis(panel, epoch, *combination))
    return pd.DataFrame(entropies)

def plot_histogram_for_joint_entropies(entropies):
    entropies.hist()
    plt.show()
    
def mean_of_entropies(entropies):
    return entropies.mean()

panel = pd.Panel(loadmat('data/P01.mat')['data'])
entropies = joint_entropy_for_subject(panel)
print(entropies)
plot_histograms_for_subject(entropies)
mean_of_entropies(entropies)

In [ ]:
from os import listdir
from os.path import join

files_S = [f for f in listdir('data') if (f.endswith('.mat') and f.startswith('S'))]
files_P = [f for f in listdir('data') if (f.endswith('.mat') and f.startswith('P'))]
global_means_for_S = []
global_means_for_P = []

for filename in files_S:
    mat = loadmat(join('data', filename))
    panel = pd.Panel(mat['data'])
    entropies = joint_entropy_for_subject(panel)
    global_mean = mean_of_entropies(entropies)
    print("Subject: {}, Entropy: {}".format(filename, global_mean))
    global_means_for_S.append(global_mean)
    del mat
    del panel
    gc.collect()
    
for filename in files_P:
    mat = loadmat(join('data', filename))
    panel = pd.Panel(mat['data'])
    entropies = joint_entropy_for_subject(panel)
    global_mean = mean_of_entropies(entropies)
    print("Subject: {}, Entropy: {}".format(filename, global_mean))
    global_means_for_P.append(global_mean)
    del mat
    del panel
    gc.collect()